In [1]:
import torch
from torch import Tensor
from torch.utils import data as torch_data
from torch.amp import autocast, GradScaler

from models import UNet
from datasets import PreprocessedOpenFWI

In [2]:
# model
model = UNet(
    in_channels=5,
    out_channels=1,
    start_features=32,
    depth=4
)
# data
train_dataset = PreprocessedOpenFWI(train=True, norm_output=True, nb_files_to_load=1)
train_loader = torch_data.DataLoader(train_dataset, batch_size=128, shuffle=True)
test_dataset = PreprocessedOpenFWI(train=False, norm_output=True)
test_loader = torch_data.DataLoader(train_dataset, batch_size=128, shuffle=True)
# Training
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)
scaler = GradScaler(device="cpu")

Output()

Output()

Output()

Output()

In [ ]:
step = 0
for epoch in range(2):
    total_loss = 0
    total_accuracy = 0
    # Count nb samples instead of accessing len(data_loader.dataset)
    # in case the data lauder augments the number of samples
    nb_batches = 0
    nb_samples = 0
    for batch_x, batch_y in train_loader:
        nb_batches += 1
        nb_samples += len(batch_x)
        model.train()
        optimizer.zero_grad()
        # print(batch_x.dtype)
        with autocast(device_type="cpu"):
            batch_y_pred = model(batch_x)

        loss_value = (batch_y_pred - batch_y).abs().mean()
        scaler.scale(loss_value).backward()
        # if self.grad_clip:
        #     utils.clip_grad_value_(model.parameters(), self.grad_clip)
        scaler.unscale_(optimizer)
        scaler.step(optimizer)
        scaler.update()

        total_loss += loss_value.item()
        # total_accuracy += (torch.max(batch_y_pred, 1)[1] == batch_y).sum().item()
        # self.optimizer.step()
        
        # if self.lr_scheduler:
        #     self.lr_scheduler.step()
        step += 1
        print("loss:", total_loss / len(train_loader))


loss: 0.126109778881073
loss: 0.2326793670654297
